# Introduction à la fouille de texte en Python: la densité lexicale

## Introduction

Maintenant qu'on a vu les bases de Python, on va avancer un peu vers de l'analyse computationnelle plus poussée.

Il s'agira de comparer trois romans de Virginia Woolf: *Mrs. Dalloway* (1925), *To the lighthouse* (1927), *The Waves* (1931). On va voir si le style de Woolf évolue, et si oui, comment. On analyse les textes par "lecture distante" (par opposition à une lecture "proche", humaine): on ne *lit pas* les romans, mais on les prend comme corpus de données qui peuvent être approchées par des méthodes computationnelles. On peut donc travailler sur de grands corpus, ici de trois romans entiers:

- **Mrs. Dalloway** décrit la journée de Clarissa Dalloway, une femme britannique de classe sociale supérieure, avant la 1e Guerre Mondiale. Le roman suit le flux de conscience de Clarissa, et il est souvent considéré comme une réponse à *Ulysses* de Joyce, qui suit lui aussi un personnage pendant une journée (livre qu'elle critique abondamment par ailleurs).
- **To the lighthouse** décrit quelques journées de la famille Ramsay et de leurs invité.e.s dans leur maison de villégiature, sur l'île de Syke en Écosse. Le roman s'étale sur une période de 10 ans, avant et après la Première Guerre Mondiale. Il entrecroise différentes histoires familiales, contient peu de dialogues directs et se concentre sur les pensées des personnages.
- **The Waves** est composé de solliloques et dialogues de six narrateur.ice.s qui s'étalent sur plusieurs années, de l'enfance à l'âge adulte. C'est généralement considéré comme son roman le plus expérimental, et le plus difficile à lire.

Notre question de recherche:

> Le style de Woolf évolue-t-il d'un roman à l'autre?
> Est-ce que le caractère "expérimental" et "difficile" de *The Waves* ressort d'une analyse statistique ?

Le choix du corpus, et une partie de la méthodologie, sont inspirés de:

> Hussein, K. & Kadhim, R. (2020). A Corpus-Based Stylistic Identification of Lexical Density Profile of Three Novels by Virginia Woolf: The Waves, Mrs. Dalloway and To the Lighthouse. *International Journal of Psychosocial Rehabilitation*. 24. pp. 6688-9702. En accès libre à [cette addresse](https://www.researchgate.net/publication/343797320_A_Corpus-Based_Stylistic_Identification_of_Lexical_Density_Profile_of_Three_Novels_by_Virginia_Woolf_The_Waves_Mrs_Dalloway_and_To_the_Lighthouse)

---
---

## Le programme des festivités

### Pipeline

Ces trois romans ont été téléchargés depuis Archive.org en texte brut (c'est-à-dire, sans éléments de mise en page) et légèrement nettoyés (les en-têtes, fin de page, la pagination et la séparation de chapitres sont supprimés). La chaîne de traitement est la suivante:

- **on ouvre les fichiers** et on lit leur contenu
- **on les simplifie** rapidement
- on découpe le texte en **liste de paragraphes** et on étudie la structure de chaque paragraphe
- on découpe le texte en **liste de phrases** et on étudie la structure de chaque phrase
- enfin, on étudie le **vocabulaire**: sa *densité lexicale*, sa distribution dans les trois textes.

On commencera par des opérations basiques en Python, pour on finira par introduire quelques techniques de base en traitement automatisé du langage (TAL).

### Compétences

Notre analyse permet d'introduire au TAL et aux bases de l'utilisation de Python pour la fouille de texte:

- **les bases de la syntaxe** Python
- **manipulation des types de données basiques** en Python: chaînes de caractères, nombres entiers et décimaux, listes et dictionnaires
- **expressions régulières** (`regex`) pour détecter des motifs dans du texte
- **traitement automatisé du langage, avec `nltk`** (enfin, une très très très rapide introduction à quelques concepts de TAL)
- **utilisation de librairies** Python
- **calculs statistiques basiques**

---
---

## Les librairies

On commence par installer les **librairies** nécessaires. 

> En Python, certaines fonctionnalités sont disponibles de base, mais la plupart sont organisées dans des libraires.
> - Une librairie est un ensemble de fonctions rassemblées ensemble avec un but spécifique: visualisations, calcul...
> - Certaines librairies sont disponibles de base en Python, d'autres sont développées par des tiers et doivent être installées. Une librairie peut aussi être appelée "module" ou "package".

Les fichiers sont installés avec la commande `pip install <package name>` dans un terminal, ou `!pip install` dans un notebook.

In [1]:
!pip install click==8.1.7
!pip install contourpy==1.1.1
!pip install cycler==0.12.1
!pip install fonttools==4.49.0
!pip install importlib-resources==6.1.2
!pip install joblib==1.3.2
!pip install kiwisolver==1.4.5
!pip install matplotlib==3.7.5
!pip install nltk==3.8.1
!pip install numpy==1.24.4
!pip install packaging==23.2
!pip install pillow==10.2.0
!pip install pyparsing==3.1.1
!pip install python-dateutil==2.9.0.post0
!pip install pytz==2024.1
!pip install regex==2023.12.25
!pip install six==1.16.0
!pip install tabulate==0.9.0
!pip install tqdm==4.66.2
!pip install tzdata==2024.1
!pip install Unidecode==1.3.8
!pip install zipp==3.17.0

  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached contourpy-1.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (301 kB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached fonttools-4.49.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
  Using cached importlib_resources-6.1.2-py3-none-any.whl (34 kB)
  Attempting uninstall: importlib-resources
    Found existing installation: importlib-resources 6.4.5
    Uninstalling importlib-resources-6.4.5:
      Successfully uninstalled importlib-resources-6.4.5
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached kiwisolver-1.4.5-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
  Using cached matplotlib-3.7.5-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.2 MB)
     |████████████████████████████████| 104 kB 9.3 MB/s eta 0:00:01
     |█████████████

Ensuite, on importe les libraires nécessaires pour les utiliser dans notre chaîne de traitement.

In [2]:
from nltk.corpus import stopwords
from unidecode import unidecode
from tabulate import tabulate
import statistics
import random
import nltk
import os
import re

Pour importer une librairie, on utilise la commande `import`:

```python
import nom_du_paquet
```

Les librairies sont souvent organisées en *sous-modules* (comme les chapitres d'un livre). Il est donc aussi possible d'importer seulement un *module*, ou une seule fonction:

```python
from nom_du_paquet import nom_du_module
from nom_du_paquet.nom_du_module import nom_du_module_ou_fonction
```

Ensuite, `nltk` (la librairie faite pour le traitement du langage naturel) demande de faire quelques téléchargements:



In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/paulhector/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/paulhector/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/paulhector/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/paulhector/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/paulhector/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

---

# Lire les fichiers

(Attention, il faut que les [trois fichiers](https://github.com/paulhectork/cours_ens2024_fouille_de_texte/tree/main/in) aient été téléchargés au format `.txt` dans la partie `Fichiers` de Google Collab, sans les renommer !)

On commence donc par lire le contenu de nos textes. Comme on a 3 textes sur lesquels on va vouloir mener les mêmes opérations, on définit des fonctions.

In [4]:
def lire(nom_de_fichier):
    with open(nom_de_fichier, mode="r" ) as fh:
        corpus = fh.read()
    return corpus

In [6]:
# qu'est-ce qui se passe ici?
dalloway = lire("in_mrs_dalloway.txt")
lighthouse = lire("in_to_the_lighthouse.txt")
waves = lire("in_the_waves.txt")

print(waves)

The sun bad not yet risen. The sea was indistinguishable from the sky, except that the sea was slightly creased as if a cloth bad wrinkles in it. Gradually as the sky whitened a dark line lay on the horizon dividing the sea from the sky and the grey cloth became barred wito thick strokes moving, one after another, beneath the surface, following each other, pursuing each other, perpetually. 

As they neared the shore each bar rose, heaped itself, broke and swept a thin veil of white water across the sand. The wave paused, 2nd then drew out again, sighing like a sleeper whose breath comes and goes unconsciously. Gradually the dark bar on the horizon became clear as if the sediment in an old wine-bottle bad sunk and left the glass green. Behind it, too, the sky cleared as if the white sediment there had sunk, or as if the arm of a woman couched beneath the horizon had raised a lamp and flat bars of white, green and yellow spread across the sky like the blades of a fan. Then she raised her

In [ ]:
def simplify(txt):
    """
    simplifier les 3 romans
    """
    txt = txt.lower() # supprimer les majuscules

    # supprimer les points qui ne séparent pas 2 phrases
    txt = txt.replace("mrs.", "mrs")  # syntaxe: `replace(<texte à remplace>, <remplacement>)`
    txt = txt.replace("ms.", "ms")
    txt = txt.replace("mr.", "mr")
    txt = txt.replace("dr.", "dr")
    txt = txt.replace("'s", "")
    # on peut l'écrire d'autres manières:
    # txt = txt.replace("mrs.", "mrs").replace("ms.", "ms").replace("mr.", "mr").replace("dr.", "dr").replace("'s", "")

    # supprimer les accents des lettres accentuées
    txt = unidecode(txt)  # unidecode n'existe pas par défaut, cette fonction a été installée avec la librairie `unidecode`

    return txt

waves = simplify(waves)
dalloway = simplify(dalloway)
lighthouse = simplify(lighthouse)

print(lighthouse)


---

# Étudier le vocabulaire: la *densité lexicale*

## Définitions

Jusqu'à maintenant, on est resté sur des méthodes d'analyse assez basiques. C'est vraiment moins de l'analyse que de la restructuration du texte. Ça nous a permis d'introduire quelques bases de Python, mais on est assez loin de produire quelque chose de vraiment intéressant.

À partir de maintenant, on va utiliser des méthodes de TAL, avec la librairie Python dédiée, `nltk`. Il existe tout plein de méthodes de traitement du langage (reconnaissance d'entités nommées, *topic modelling*, *sentiment analysis*...)

L'avantage d'une librairie, c'est que ça prémache complètement le travail. Le désavantage, c'est qu'il faut savoir utiliser une librairie (quelles fonctions utiliser, comment etc.). Pour savoir comment les utiliser, on se réfère à la [documentation](https://www.nltk.org/api/nltk.tokenize.word_tokenize.html). La documentation, c'est parfois obscur. Apprendre à lire de la doc c'est donc un travail à part entière quand on apprend à programmer.

Manipulons rapidement le texte avec deux fonctions de `nltk`:

In [9]:
# tokenisation au mot (similaire à txt.split(" "), mais performe des simplifications en plus)
tokens = nltk.word_tokenize(waves)
print(tokens)
print(type(tokens))

# tokens est une liste. comment affichier le nombre de tokens dans the waves ?

['The', 'sun', 'bad', 'not', 'yet', 'risen', '.', 'The', 'sea', 'was', 'indistinguishable', 'from', 'the', 'sky', ',', 'except', 'that', 'the', 'sea', 'was', 'slightly', 'creased', 'as', 'if', 'a', 'cloth', 'bad', 'wrinkles', 'in', 'it', '.', 'Gradually', 'as', 'the', 'sky', 'whitened', 'a', 'dark', 'line', 'lay', 'on', 'the', 'horizon', 'dividing', 'the', 'sea', 'from', 'the', 'sky', 'and', 'the', 'grey', 'cloth', 'became', 'barred', 'wito', 'thick', 'strokes', 'moving', ',', 'one', 'after', 'another', ',', 'beneath', 'the', 'surface', ',', 'following', 'each', 'other', ',', 'pursuing', 'each', 'other', ',', 'perpetually', '.', 'As', 'they', 'neared', 'the', 'shore', 'each', 'bar', 'rose', ',', 'heaped', 'itself', ',', 'broke', 'and', 'swept', 'a', 'thin', 'veil', 'of', 'white', 'water', 'across', 'the', 'sand', '.', 'The', 'wave', 'paused', ',', '2nd', 'then', 'drew', 'out', 'again', ',', 'sighing', 'like', 'a', 'sleeper', 'whose', 'breath', 'comes', 'and', 'goes', 'unconsciously', '

In [11]:
# pos_tag = part-of-speech tagging. on verra se qui se passe plus en détail plus tard
print( nltk.pos_tag(tokens, tagset="universal") )

# est-ce que vous avez une idée de ce qui s'est passé au dessus ?

[('The', 'DET'), ('sun', 'NOUN'), ('bad', 'ADJ'), ('not', 'ADV'), ('yet', 'ADV'), ('risen', 'VERB'), ('.', '.'), ('The', 'DET'), ('sea', 'NOUN'), ('was', 'VERB'), ('indistinguishable', 'ADJ'), ('from', 'ADP'), ('the', 'DET'), ('sky', 'NOUN'), (',', '.'), ('except', 'ADP'), ('that', 'ADP'), ('the', 'DET'), ('sea', 'NOUN'), ('was', 'VERB'), ('slightly', 'ADV'), ('creased', 'VERB'), ('as', 'ADP'), ('if', 'ADP'), ('a', 'DET'), ('cloth', 'NOUN'), ('bad', 'ADJ'), ('wrinkles', 'NOUN'), ('in', 'ADP'), ('it', 'PRON'), ('.', '.'), ('Gradually', 'ADV'), ('as', 'ADP'), ('the', 'DET'), ('sky', 'NOUN'), ('whitened', 'VERB'), ('a', 'DET'), ('dark', 'ADJ'), ('line', 'NOUN'), ('lay', 'NOUN'), ('on', 'ADP'), ('the', 'DET'), ('horizon', 'NOUN'), ('dividing', 'VERB'), ('the', 'DET'), ('sea', 'NOUN'), ('from', 'ADP'), ('the', 'DET'), ('sky', 'NOUN'), ('and', 'CONJ'), ('the', 'DET'), ('grey', 'ADJ'), ('cloth', 'NOUN'), ('became', 'VERB'), ('barred', 'ADJ'), ('wito', 'NOUN'), ('thick', 'NOUN'), ('strokes', '

Vous l'aurez vu, avec le [`POS tag`](https://fr.wikipedia.org/wiki/%C3%89tiquetage_morpho-syntaxique) (part-of-speech tagging, ou étiquetage morphosyntaxique), on peut classer les mots du texte en fonction du rôle qu'ils occupent: verbe, déterminants, etc. Ça permet des analyses beaucoup plus détaillées, puisqu'on a un tagging assez fin de chaque mot. Par exemple, si on veut s'intéresser à l'usage des déterminants, on peut ne sélectionner que les *tokens* taggués avec `DET`.

L'argument `tagset` permet de préciser les types de tags à produire. *Universal* définit un jeu de tags assez générique (il en existe des plus précis, mais inutiles ici):

```
Tag  | Meaning             | English Examples
~~~~~|~~~~~~~~~~~~~~~~~~~~~|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ADJ  | adjective           | new, good, high, special, big, local
ADP  | adposition          | on, of, at, with, by, into, under
ADV  | adverb              | really, already, still, early, now
CONJ | conjunction         | and, or, but, if, while, although
DET  | determiner, article | the, a, some, most, every, no, which
NOUN | noun                | year, home, costs, time, Africa
NUM  | numeral             | twenty-four, fourth, 1991, 14:24
PRT  | particle            | at, on, out, over per, that, up, with
PRON | pronoun             | he, their, her, its, my, I, us
VERB | verb                | is, say, told, given, playing, would
.    | punctuation marks   | . , ; !
X    | other               | ersatz, esprit, dunno, gr8, univeristy
```

On peut maintenant estimer la proportion de verbes dans le texte, par exemple:

In [13]:
tokens_tags = nltk.pos_tag(tokens, tagset="universal")

verb_count = 0
for (token, tag) in tokens_tags:
    if tag == "VERB":
        verb_count += 1
print(verb_count)
        
# et maintenant, comment est-ce qu'on calcule la proportion de verbes ? 

15356


Ce calcul au dessus, c'est la base de la [densité lexicale](https://en.wikipedia.org/wiki/Lexical_density) ! 

La densité lexicale vise à mesurer la complexité informationnelle d'un texte, écrit ou oral. Elle s'appuie sur la classification des mots en deux catégories:
- les **mots-lexicaux** (`lexical units`) regroupent l'ensemble de mots "porteurs d'information" dans un texte: les noms, les verbes, les adverbes et les adjectifs qualitatifs. On considère que c'est grâce à eux que l'information est transmise.
- les **mots-outils ou mots-grammaticaux** (`function words`) regroupent tout le reste des mots. On considère qu'ils sont moins porteurs de sens et qu'ils servent surtout à la structure de la phrase

La classification entre ces deux groupes n'est pas clairement définie et peut varier d'une étude à l'autre.

La **densité lexicale** (notée $L_d$) mesure, sur une échelle de 0 à 100, la proportion de mots lexicaux dans un texte. 

Elle se mesure en  calculant la proportion de *mots lexicaux* par rapport au nombre total de mots. Soit $N_{lex}$ le nombre de mots lexicaux et $N$ le nombre total de mots dans un texte:

$$L_d = (\frac{N_{lex}}{N}) \times 100$$

Plus $L_d$ est proche de 100, plus il y a de mots lexicaux, plus on considère que le texte est riche en information. On considère en moyenne que que:
- pour un texte écrit, $L_d > 40$, et pour un texte oral, $L_d < 40$.
- pour un texte de fiction, $ 40 < L_d < 54$; pour un texte de non-fiction, $40 < L_d < 65$.

PS: il existe plusieurs variantes de calcul, mais on utilise celle définie par Ure J. (1971).

### Chaîne de traitement

Avec `nltk`, calculer une densité lexicale est vraiment très très simple:

- on tokenise notre texte en `tokens` (items lexicaux)
- on fait un *POS tag*
- on compte le nombre d'occurrences pour chaque fonction des tokens d'un texte
- à partir de là, on calcule notre $L_d$

---

Pour commencer, on généralise ce qui avait déjà été fait au dessus: on avait travaillé seulement sur les verbes, maintenant on travaille sur tout.

In [16]:
# on reprend le POS tagging
tokens = nltk.word_tokenize(waves)  # tokenisation au mot (similaire à txt.split(" "), mais performe des simplifications en plus)
print(tokens[:10])

size = len(tokens)                              # la taille du corpus
pos = nltk.pos_tag(tokens, tagset="universal")  # part-of-speech tagging. universal définit des classes très généralistes
print(pos[:10])                                 # que voit-on ici?
print(size)

['The', 'sun', 'bad', 'not', 'yet', 'risen', '.', 'The', 'sea', 'was']
[('The', 'DET'), ('sun', 'NOUN'), ('bad', 'ADJ'), ('not', 'ADV'), ('yet', 'ADV'), ('risen', 'VERB'), ('.', '.'), ('The', 'DET'), ('sea', 'NOUN'), ('was', 'VERB')]
92454


Ensuite, on regarde le nombre d'occurrences de chaque tag. On pourrait faire un dict qui associe chaque tag à son nombre d'occurrences, mais NLTK mâche le travail en faisant des distributions de fréquences à partir de listes de valeurs.

> En statistiques, une distribution de fréquences, c'est un tableau qui associe les différentes valeurs d'un échantillon à la fréquence à laquelle ces valeurs apparaissent.


In [17]:
tags = []
for (token, tag) in pos:
    tags.append(tag)

fd = nltk.FreqDist(tags)  # `nltk.FreqDist()` associe à chaque valeur distincte d'une liste le nombre d'occurrences dans cette liste
fd.tabulate()

 NOUN  VERB     .   ADP   DET  PRON   ADJ   ADV  CONJ   PRT   NUM     X 
19522 15356 13719  9588  9252  8913  5292  4698  3432  1963   651    68 


On est presque prêt.e.s ! Il suffit maintenant d'additionner toutes les fréquences de toutes les unités lexicales (noms, verbes...) et de comparer au reste du corpus. 

In [20]:
# fd.get() permet de récuperer la fréquence de la valeur entre parenthèses
nlex = fd.get("NOUN") + fd.get("VERB") + fd.get("ADJ") + fd.get("ADV")  # nb d'unités lexicales

print("taille du corpus          :", size)
print("nombre de mots-outils     :", size - nlex)
print("nombre d'unités lexicales :", nlex)

taille du corpus          : 92454
nombre de mots-outils     : 47586
nombre d'unités lexicales : 44868


In [22]:
ld = (nlex/size) * 100
print(ld)

48.53007982347978


On a maintenant la densité lexicale de *The Waves* ! On écrit une fonction qui reprenne tout ça et on la lance:

In [24]:
def densite_lexicale(text):
    """
    enfin, on étudie la densité lexicale de chaque roman

    on suit la méthode de Ure: 100 * <nb d'unités lexicales> / <nb de tokens>
    https://en.wikipedia.org/wiki/Lexical_density
    https://www.nltk.org/book/ch05.html
    """
    tokens = nltk.word_tokenize(text)    # tokenisation au mot (similaire à txt.split(" "), mais performe des simplifications en plus)

    size = len(tokens)  # on travaille sur tout le corpus
    tokens_tags = nltk.pos_tag(tokens, tagset="universal")  # part-of-speech tagging (classification du texte en classes: verbes...). universal définit des classes très généralistes
    tags = []
    for (token, tag) in tokens_tags:
        tags.append(tag)

    fd = nltk.FreqDist(tags)  # valeur associée aux nombre d'occurrences de celle-ci
    nlex = fd.get("NOUN") + fd.get("VERB") + fd.get("ADJ") + fd.get("ADV")  # nb d'unités lexicales
    ld = 100 * (nlex/size)

    return ld


ld_lighthouse = densite_lexicale(lighthouse)
ld_dalloway = densite_lexicale(dalloway)
ld_waves = densite_lexicale(waves)

print("Mrs Dalloway      (1925):", ld_dalloway)
print("To the lighthouse (1927):", ld_lighthouse)
print("The Waves         (1931):", ld_waves)


Mrs Dalloway      (1925): 48.22437449556094
To the lighthouse (1927): 47.12100710385154
The Waves         (1931): 48.53007982347978


In [ ]:
headers = [ "", "Mrs. Dalloway, 1925", "To the Lighthouse, 1927", "The Waves, 1931" ]
data = [ [ "nombre médian de mots par paragraphes"
         , stats_dalloway["nombre médian de mots par paragraphes"]
         , stats_lighthouse["nombre médian de mots par paragraphes"]
         , stats_waves["nombre médian de mots par paragraphes"]
         ],
         [ "nombre médian de phrases par paragraphes"
         , stats_dalloway["nombre médian de phrases par paragraphes"]
         , stats_lighthouse["nombre médian de phrases par paragraphes"]
         , stats_waves["nombre médian de phrases par paragraphes"]
         ],
         [ "nombre médian de mots par phrase"
         , stats_dalloway["nombre médian de mots par phrase"]
         , stats_lighthouse["nombre médian de mots par phrase"]
         , stats_waves["nombre médian de mots par phrase"]
         ],
         [ "nombre moyen de signes de ponctuation par phrase"
         , stats_dalloway["nombre moyen de signes de ponctuation par phrase"]
         , stats_lighthouse["nombre moyen de signes de ponctuation par phrase"]
         , stats_waves["nombre moyen de signes de ponctuation par phrase"]
         ],
         [ "densité lexicale"
         , stats_dalloway["densité lexicale"]
         , stats_lighthouse["densité lexicale"]
         , stats_waves["densité lexicale"]
         ]
]
print(tabulate(data, headers, tablefmt="rounded_grid"))

Qu'est-ce qu'on peut dire de ces résultats?

In [ ]:
txt = lighthouse
print(txt[:100])

# 1)
# `[^<motif>]` veut dire "tout sauf <motif>".
# `[^a-z ]` veut donc dire "tout ce qui n'est pas un espace ou un caractère alphabétique"
txt  = re.sub("[^a-z ]", " ", txt)  # que fait donc cette ligne?
txt = re.sub("\s+", " ", txt)       # on normalise les espaces
print(txt[:100])

# 2) on tokenise
tokens = nltk.word_tokenize(txt)
print(tokens[:100])
print(len(tokens))

### Étape 3
Ensuite, **on supprime les stopwords**


In [ ]:
# 3) on supprime les stopwords
stop_words = list(stopwords.words("english"))
print(stop_words[:10])

tokens_filtered = []
for token in tokens:
    if token.lower() not in stop_words:
        tokens_filtered.append(token)
print(tokens_filtered[:100])
print(len(tokens_filtered))

# en une ligne, comment calculer le nombre de tokens qui ont été supprimées?

### Étape 4
Pour continuer, on fait le **`pos tagging`et on crée notre corpus de 5000 mots-contenu**.

In [ ]:
size = 5000  # la taille du corpus final: 5000 tokens
sample_pos = []
pos = nltk.pos_tag(tokens_filtered, tagset="universal")  # part-of-speech tagging (classification du texte en classes: verbes...). universal définit des classes très généralistes
for (token, tag) in pos:
    if tag in [ "NOUN", "VERB", "ADJ", "ADV" ]:  # si ce token est un mot-contenu
        sample_pos.append(token)
sample_pos = random.sample(sample_pos, size)  # on créé une liste de 5000 items créés au hasard dans `sample_pos`

print(sample_pos[:100])
print(len(sample_pos))

### Étape 5

**On lemmatise**. Comme pour le `pos tagging`, `nltk` propose des outils tout prêts, et plusieurs lemmatiseurs. On utilise le `WordNetLemmatizer`, entraîné sur le jeu de données [`WordNet`](https://wordnet.princeton.edu/)**texte en gras**

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
sample_lem = [ lemmatizer.lemmatize(token) for token in sample_pos ]  # exemple d'itération en une ligne: on boucle sur tous les items de `sample_pos` et on leur applique une fonction
print(sample_lem)
print(len(set(sample_lem)))  # le nombre de lemmes uniques dans notre liste de 5000 lemmes (`set` est un type de données équivalent à une ligne, mais sans doublons. utiliser `len()` sur une liste permet donc de supprimer les doublons de la `list`)

### Étape 6

On calcule notre distribution de fréquences. C'est tout simple, comme on l'a déjà vu:

In [ ]:
fd = nltk.FreqDist(sample_lem)
fd.tabulate()  # les 10 lemmes les plus utilisés

fd.plot(title="To the lighthouse")  # ça prend un peu de temps et c'est pas nécessairement ultra lisible

## Étape 7

Comme on l'a vu avec le graphique ci-dessus, pour comparer les distributions de fréquences entre nos 3 romans, pour le moment, c'est pas donné: la courbe contient plusieurs milliers de valeurs, ce qui rend l'analyse à l'oeil nu difficile.

On va donc **classer les lemmes en 10 groupes**, des 10% des lemmes les plus utilisés au 10% des lemmes les moins utilisés. C'est la partie la plus compliquée du code, mais elle ne comporte que des choses que l'on a déjà vues !

In [ ]:
distribution_moyenne = []  # [ <moyenne d'occurrences pour un lemme, par décile> ]
distribution_somme = []    # [ <nombre absolu de lemmes, par décile> ]

for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:  # on itère sur chaque décile
    k = []  # liste du nombre d'occurrences pour chaque lemme, pour le décile actuel

    # on remplit `k` avec le nombre d'occurrences par quartile
    for mot, occurrences in fd.items():          # `fd.items()` permet de boucler sur des couples `<lemme>`: `<nombre d'occurrences>`
        print(mot, occurrences)

        quantile = occurrences/max(fd.values())  # représentation de l'utilisation du mot sur une échelle 0..1: 0 = mot jamais utilisé, 1 = mot le plus utilisé

        # on vérifie si `quantile` correspond au quantile actuel `i`.
        # si oui, on ajoute à `k`, notre liste du nombre d'occurrences,
        # le nombre d'occurrences de ce lemme
        if i != 1:
            if i > quantile >= i-0.1:
                k.append(occurrences)
        else:
            if i >= quantile >= i-0.1:
                k.append(occurrences)

    # on a fini de remplir `k` pour ce décile avec tous les lemmes du roman
    # on calcule nos statistiques et on les ajoute aux listes `distribution`
    if len(k) > 0:
        mean = round(statistics.mean(k), 3)
        distribution_moyenne.append(mean)
    else:
        distribution_moyenne.append(0)  # 0 mot ne rentre dans ce quantile => on ne peut calculer de moyenne
    distribution_somme.append(len(k))

print(distribution_somme)
print(distribution_moyenne)
print(distribution_moyenne[-1])  # comment interpréter ce nombre?
print(distribution_somme[2])      # et celui-ci?

Et voilà ! Maintenant, on fait de tout ça une fonction, et ajoute les résultats à nos dictionnaires de statistiques.

In [ ]:
def distribution_vocabulaire(txt, stats, title):
    """
    étudier la distribution du vocabulaire dans les trois romans
    """
    # étape 1
    txt = re.sub("[^a-z ]", " ", txt)  # on enlève tous les caractères non-alphabétiques et les espaces
    txt = re.sub("\s+", " ", txt)      # on normalise les espaces

    # étape 2
    tokens = nltk.word_tokenize(txt)   # tokenisation au mot (similaire à txt.split(" "), mais performe des simplifications en plus)

    # étape 3
    # on supprime tous les stopwords (mots jugés
    # "inutiles" pour l'analyse automatique)
    tokens_filtered = []
    stop_words = set(stopwords.words("english"))
    for token in tokens:
        if token.lower() not in stop_words:
            tokens_filtered.append(token)

    # étape 4
    # on fait un part-of-speech tagging sur le
    # corpus pour pouvoir ensuite le lemmatiser
    size = 5000  # la taille du corpus final: 5000 tokens
    sample_pos = []
    pos = nltk.pos_tag(tokens_filtered, tagset="universal")  # part-of-speech tagging (classification du texte en classes: verbes...). universal définit des classes très généralistes
    for (token, tag) in pos:
        if tag in [ "NOUN", "VERB", "ADJ", "ADV" ]:
            sample_pos.append(token)
    sample_pos = random.sample(sample_pos, size)

    # étape 5
    # on lemmatise le corpus
    lemmatizer = nltk.stem.WordNetLemmatizer()
    sample_lem = [ lemmatizer.lemmatize(token) for token in sample_pos ]

    # étape 6
    # calculter une distribution de fréquences
    fd = nltk.FreqDist(sample_lem)  # mot / nombres d'occurrences
    fd.plot(title=title)

    # étape 7
    # grouper le vocabulaire en quantiles:
    # lecture: les 10% des lemmes les plus fréquemment
    # rencontrés sont utilisés en moyenne n fois.
    distribution_moyenne = []  # [ <moyenne d'occurrences pour un lemme, par décile>]
    distribution_somme = []    # [ <nombre absolu de lemmes, par décile>]
    for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        k = []  # liste du nombre d'occurrences par quantile
        # on remplit `k` avec le nombre d'occurrences par quartile
        for mot, occurrences in fd.items():
            quantile = occurrences/max(fd.values())  # représentation de l'utilisation du mot sur une échelle 0..1: 0 = mot jamais utilisé, 1 = mot le plus utilisé
            if i != 1:
                if i > quantile >= i-0.1:
                    k.append(occurrences)
            else:
                if i >= quantile >= i-0.1:
                    k.append(occurrences)
        # on calcule nos statistiques et on les ajoute aux listes `distribution`
        if len(k) > 0:
            mean = round(statistics.mean(k), 3)
            distribution_moyenne.append(mean)
        else:
            distribution_moyenne.append(0)  # 0 mot ne rentre dans ce quantile => on ne peut calculer de moyenne
        distribution_somme.append(len(k))

    # enfin, on crée des dictionnaires pour afficher les résultats de façon lisible
    deciles = [ "0..10", "10..20", "20..30", "30..40", "40..50"
              , "50..60", "60..70", "70..80", "80..90", "90..100" ]
    somme = {}
    moyenne = {}
    for i,d in enumerate(deciles):  # enumerate permet d'itérer sur une liste avec un couple un couple [index, valeur]
        somme[d] = distribution_somme[i]
        moyenne[d] = distribution_moyenne[i]
    stats["lemmes distincts"] = somme
    stats["lemme moyenne"] = moyenne
    return stats

stats_lighthouse = distribution_vocabulaire(lighthouse, lighthouse_stats, "To the lighthouse")
stats_dalloway = distribution_vocabulaire(dalloway, dalloway_stats, "Mrs. Dalloway")
stats_waves = distribution_vocabulaire(waves, stats_waves, "The Waves")


In [ ]:
headers = [ "", "Mrs. Dalloway, 1925", "To the Lighthouse, 1927", "The Waves, 1931" ]
data = [ [ "nombre médian de mots par paragraphes"
         , stats_dalloway["nombre médian de mots par paragraphes"]
         , stats_lighthouse["nombre médian de mots par paragraphes"]
         , stats_waves["nombre médian de mots par paragraphes"]
         ],
         [ "nombre médian de phrases par paragraphes"
         , stats_dalloway["nombre médian de phrases par paragraphes"]
         , stats_lighthouse["nombre médian de phrases par paragraphes"]
         , stats_waves["nombre médian de phrases par paragraphes"]
         ],
         [ "nombre médian de mots par phrase"
         , stats_dalloway["nombre médian de mots par phrase"]
         , stats_lighthouse["nombre médian de mots par phrase"]
         , stats_waves["nombre médian de mots par phrase"]
         ],
         [ "nombre moyen de signes de ponctuation par phrase"
         , stats_dalloway["nombre moyen de signes de ponctuation par phrase"]
         , stats_lighthouse["nombre moyen de signes de ponctuation par phrase"]
         , stats_waves["nombre moyen de signes de ponctuation par phrase"]
         ],
         [ "densité lexicale"
         , stats_dalloway["densité lexicale"]
         , stats_lighthouse["densité lexicale"]
         , stats_waves["densité lexicale"]
         ],
         [ "distribution du vocabulaire\n(nombre de lemmes distincts par décile)"
         , "\n".join(f"{k} : {v}" for k,v in stats_dalloway["lemmes distincts"].items() )
         , "\n".join(f"{k} : {v}" for k,v in stats_lighthouse["lemmes distincts"].items() )
         , "\n".join(f"{k} : {v}" for k,v in stats_waves["lemmes distincts"].items() )
         ],
         [ "distribution du vocabulaire\n(moyenne d'utilisation d'un lemme par décile)"
         , "\n".join(f"{k} : {v}" for k,v in stats_dalloway["lemme moyenne"].items() )
         , "\n".join(f"{k} : {v}" for k,v in stats_lighthouse["lemme moyenne"].items() )
         , "\n".join(f"{k} : {v}" for k,v in stats_waves["lemme moyenne"].items() )
         ]
]
print(tabulate(data, headers, tablefmt="rounded_grid"))


Les résultats ne sont pas *criants*, mais on voit quand même clairement que *To the lighthouse* marque un net déclin dans la densité lexicale, et que *The Waves*, censé être le plus difficile et expérimental des trois livres, a une densité lexicale supérieure. À mon humble avis, *To the lighthouse* n'est pas une lecture évidente pour autant, même si elle vaut le coup d'être faite :) 

La densité lexicale n'est qu'une mesure, et elle est elle-même très limitée. Cependant, on voit maintenant qu'en une fonction et moins de 50 lignes de code qu'on arrive à faire une étude quantitative complète sur trois livres, de façon quasi instantannée !

---

## Pour aller plus loin

J'ai quelques autres notebooks qui peuvent être utiles:

- [Cours donné en 2024 à l'ENS](https://github.com/paulhectork/cours_ens2024_fouille_de_texte) (2h). C'est le même corpus de trois romans de Virginia Woolf, mais en plus de la densité lexicale, on voit l'analyse de la structure des phrases et paragraphes, ainsi que la distribution du vocabulaire dans les trois romans pour chercher à mener une analyse statistique un peu plus poussée.
- [Atelier donné en 2023 à l'ENS](https://github.com/paulhectork/cours_ens2023_xmltei) (1 journée) sur l'analyse de texte en XML. On y voit le XML-TEI, un standard de description de documents, et on y fait du TAL beaucoup plus avancé: géocodage, résolutions d'entités nommées... Bon ce cours est peut-être un peu compliqué mais je le mets quand même:) 


Ces ressources pédagogiques sont aussi très utiles et m'ont servi à préparer ces cours:
- Laramée, F. D. (2018). Introduction à la stylométrie en Python. *Programming historian*. [En ligne](https://programminghistorian.org/fr/lecons/introduction-a-la-stylometrie-avec-python)
- Lavin, Matthew J. (2019). Analyse de documents avec TF-IDF. *Programming historian*. [En ligne](https://programminghistorian.org/fr/lecons/analyse-de-documents-avec-tfidf)
- Bird, S. & Klein E. & Loper E. (1e édition 2009). *Natural language processing with pyton. Analyzing text with the natural language toolkit*. [En ligne](https://www.nltk.org/book/) (pour aller beaucoup plus loin) 
- [Cours de Python](https://github.com/PonteIneptique/cours-python) donné par Thibault Clérice à l'École des Chartes jusqu'en 2021-2022. Un cours trèèèèès complet qui va jusqu'au développement d'une appli Web.
